In [1]:
import threading
import time
from typing import Any, Dict

import zmq
from zmq.utils.monitor import recv_monitor_message


def line() -> None:
    print('-' * 40)


print(f"libzmq-{zmq.zmq_version()}")
if zmq.zmq_version_info() < (4, 0):
    raise RuntimeError("monitoring in libzmq version < 4.0 is not supported")

EVENT_MAP = {}
print("Event names:")
for name in dir(zmq):
    if name.startswith('EVENT_'):
        value = getattr(zmq, name)
        print("%21s : %4i" % (name, value))
        EVENT_MAP[value] = name


def event_monitor(monitor: zmq.Socket) -> None:
    while monitor.poll():
        evt: Dict[str, Any] = {}
        mon_evt = recv_monitor_message(monitor)
        evt.update(mon_evt)
        evt['description'] = EVENT_MAP[evt['event']]
        print(f"Event: {evt}")
        if evt['event'] == zmq.EVENT_MONITOR_STOPPED:
            break
    monitor.close()
    print()
    print("event monitor thread done!")


ctx = zmq.Context.instance()
rep = ctx.socket(zmq.REP)
req = ctx.socket(zmq.REQ)

monitor = req.get_monitor_socket()

t = threading.Thread(target=event_monitor, args=(monitor,))
t.start()

line()
print("bind req")
req.bind("tcp://127.0.0.1:6666")
req.bind("tcp://127.0.0.1:6667")
time.sleep(1)

line()
print("connect rep")
rep.connect("tcp://127.0.0.1:6667")
time.sleep(0.2)
rep.connect("tcp://127.0.0.1:6666")
time.sleep(1)

line()
print("disconnect rep")
rep.disconnect("tcp://127.0.0.1:6667")
time.sleep(1)
rep.disconnect("tcp://127.0.0.1:6666")
time.sleep(1)

line()
print("close rep")
rep.close()
time.sleep(1)

line()
print("disabling event monitor")
req.disable_monitor()

line()
print("event monitor thread should now terminate")

# Create a new socket to connect to listener, no more
# events should be observed.
rep = ctx.socket(zmq.REP)

line()
print("connect rep")
rep.connect("tcp://127.0.0.1:6667")
time.sleep(0.2)

line()
print("disconnect rep")
rep.disconnect("tcp://127.0.0.1:6667")
time.sleep(0.2)

line()
print("close rep")
rep.close()

line()
print("close req")
req.close()

print("END")
ctx.term()

libzmq-4.3.5
Event names:
       EVENT_ACCEPTED :   32
  EVENT_ACCEPT_FAILED :   64
            EVENT_ALL : 65535
         EVENT_ALL_V1 : 65535
         EVENT_ALL_V2 : 131071
    EVENT_BIND_FAILED :   16
         EVENT_CLOSED :  128
   EVENT_CLOSE_FAILED :  256
      EVENT_CONNECTED :    1
EVENT_CONNECT_DELAYED :    2
EVENT_CONNECT_RETRIED :    4
   EVENT_DISCONNECTED :  512
EVENT_HANDSHAKE_FAILED_AUTH : 16384
EVENT_HANDSHAKE_FAILED_NO_DETAIL : 2048
EVENT_HANDSHAKE_FAILED_PROTOCOL : 8192
EVENT_HANDSHAKE_SUCCEEDED : 4096
      EVENT_LISTENING :    8
EVENT_MONITOR_STOPPED : 1024
    EVENT_PIPES_STATS : 65536
----------------------------------------
bind req
Event: {'event': <Event.LISTENING: 8>, 'value': <Event.LISTENING|CLOSED|CLOSE_FAILED|DISCONNECTED|MONITOR_STOPPED: 1928>, 'endpoint': b'tcp://127.0.0.1:6666', 'description': 'EVENT_LISTENING'}
Event: {'event': <Event.LISTENING: 8>, 'value': <Event.CONNECT_RETRIED|LISTENING|BIND_FAILED|CLOSED|CLOSE_FAILED|DISCONNECTED|MONITOR_STOPPED: 